# Making Plots from Original images

In [1]:
#### Conda environment
# conda create --name terada2019 python=3.7
# pip install --upgrade pip
# pip install astropy scipy
# pip install photutils
# pip install jupyter matplotlib h5py aplpy pyregion PyAVM healpy

In [2]:
import os
import sys
import time
sys.path
sys.path.append('./')

import numpy as np
from astropy.stats import mad_std

from photutils import datasets
from photutils import DAOStarFinder
from photutils import aperture_photometry, CircularAperture

import aplpy
from astropy.io.fits import getdata
from astropy import wcs
from astropy.io import fits
from astropy import units as u
from astropy import constants as con
from astropy.coordinates import SkyCoord
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

import matplotlib
matplotlib.use('PDF')
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
# %matplotlib inline

#### Format of marker file:

name, ra_h ra_m ra_s dec_d dec_m dec_s  R:0-1 G:0-1 B:0-1 alpha  size

dm_tau  04 33 48.7335659850  +18 10 09.974471722  1 0.2 0.2 1.0 2.0

In [3]:
# Directory for input image
data_path   = r"../3488473-dm tau"

# Defining output directories
output_path_dict = {}
output_path_dict['R'] = './R_maps'
output_path_dict['V'] = './V_maps'
output_path_dict['I'] = './I_maps'

# set if we want to do photometry for objects in the markers list
dophotometry     = True
aperature_r_pix  = 10.0 # pixels
if (dophotometry == True):
    print("Will output photometry. Please define the output filename if not yet.")
    photometry_file_dict = {}
    photometry_file_dict['R'] = './R_phot.txt'
    photometry_file_dict['V'] = './V_phot.txt'
    photometry_file_dict['I'] = './I_phot.txt'
    

# Output ASCII list for FITS images which do not possess coordinate headers
f = open("NO_COORD_HEADER_list.txt","w")    
    
markerfile = './markers.txt'
    
# Preparation ##############################################################
for key in output_path_dict.keys():
    os.system('rm -rf ' + output_path_dict[key] )
    os.system('mkdir '  + output_path_dict[key] )
    
if (dophotometry == True):
    for key in photometry_file_dict.keys():
        os.system('rm -rf ' + photometry_file_dict[key] )
        photo_file = open(photometry_file_dict[key],"a+")
        photo_file.write("# Data  target_name  JD  Counts")

num_markers = 0
try:
    marker_label = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=0, dtype=np.str)
    rah, ram, ras = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=(1,2,3) )
    decd, decm, decs = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=(4,5,6) )
    markerR, markerG, markerB = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=(7,8,9) )
    marker_alpha = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=10)
    marker_size  = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=11)
    
    ra  = ( rah + ram / 60.0 + ras / 3600.0 ) * 15.0
    if (decd > 0 ):
        dec = decd + decm / 60.0 + decs / 3600.0
    else:
        dec = decd - decm / 60.0 - decs / 3600.0
    num_markers = np.size(ra)
    
except:
    print('No markers found')
    
images     = os.listdir( data_path )
num_images = len(images)
############################################################################
    

for i in range(0, num_images):
    image_name = images[i]
    
    info = image_name.strip('.fits').split('_')
    target_name    = info[0]
    directory_name = info[1]
    band           = info[2]
    epoch_idx      = info[3]
    
    # open FITS image
    hdulist = fits.open(data_path + '/' + image_name)
    try:
        crval1 = hdulist[0].header['crval1']
        crval2 = hdulist[0].header['crval2']
        date   = hdulist[0].header['date-obs']
        jd     = hdulist[0].header['jd']
    except:
        print('Error, coordinate header of ' + image_name + ' does not exist.')
        f.write( image_name + '\n' )
        continue

    # loading image
    if (dophotometry == True):
        try:
            image = hdulist[0].data
            w = wcs.WCS(hdulist[0].header)
        except:
            print("Error loading image " + directory_name + '_' + band + '_' + epoch_idx)
        
    
    fig = aplpy.FITSFigure(data_path + '/' + image_name)
    #fig.set_xaxis_coord_type('longitude')
    #fig.set_yaxis_coord_type('latitude')
    fig.axis_labels.hide()
    fig.show_grayscale(invert=False)
    
    # mark stars
    for j in range(0, num_markers):
        # plot markers in the png figure
        if (num_markers==1):
            x, y = ra, dec
            pixcrd2 = w.wcs_world2pix([ [x,y] ], 0)
            xpix = pixcrd2[0][0]
            ypix = pixcrd2[0][1]
            mcolor = (markerR, markerG, markerB)
            malpha = marker_alpha
            msize  = marker_size
            mlabel = str(marker_label)
        else:
            x, y = ra[j], dec[j]
            mcolor = (markerR[j], markerG[j], markerB[j])
            malpha = marker_alpha[j]
            msize = marker_size[j]
            mlabel = str(marker_label[j])
        
        fig.show_markers(
                         x, y, 
                         edgecolor=mcolor, 
                         # facecolor=facecolor[plot_id],
                         marker='o', s=msize, 
                         alpha=malpha
                        )
        fig.add_label(x, y, '  ' + mlabel, 
                      color=mcolor, fontsize=12, horizontalalignment='left')
        
        # optionally, do aperture photometry
        if (dophotometry == True):              
            # making photometry
            xpix_min = int(round(xpix - aperature_r_pix*3) )
            xpix_max = int(round(xpix + aperature_r_pix*3) )
            ypix_min = int(round(ypix - aperature_r_pix*3) )
            ypix_max = int(round(ypix + aperature_r_pix*3) )
            try:
                print(directory_name + '_' + band + '_' + epoch_idx)
                crop = image[ypix_min:ypix_max, xpix_min:xpix_max].astype(float)
                
                # background subtraction
                crop -= np.median(crop)
                
                # estimate pixel statistics
                bkg_sigma = mad_std(crop)
                
                # find stars
                daofind = DAOStarFinder(fwhm= aperature_r_pix/2.0, threshold=5.*bkg_sigma)
                sources = daofind(crop)

                
                mxcentroid_array = np.array( sources['xcentroid'] )
                mycentroid_array = np.array( sources['ycentroid'] )
                mpeak_array      = np.array( sources['peak']      )
                
                if ( len(mpeak_array) > 0 ):
                    # Very ugly code here due to unfamiliar with python.
                    # need to update. Baobab, 2019.Dec.25
                    index_array = range(0, len(mpeak_array) )
                    xindex = np.max(np.where( mpeak_array == np.max(mpeak_array), index_array, -1))
                    yindex = np.max(np.where( mpeak_array == np.max(mpeak_array), index_array, -1))
                    mxcentroid = mxcentroid_array[xindex]
                    mycentroid = mycentroid_array[yindex]
                    positions = np.transpose((mxcentroid, mycentroid))
                    apertures = CircularAperture(positions, aperature_r_pix)
                    phot_table = aperture_photometry(crop, apertures)
                    counts    = phot_table['aperture_sum'][0]

                    try:
                        photo_file = open(photometry_file_dict[band],"a+")
                        outtext     = directory_name + '_' + band + '_' + epoch_idx + '  ' + \
                                      str(marker_label) + '   ' + \
                                      str(jd).strip() + '   ' + \
                                      str(counts) + '   \n'
                        photo_file.write(outtext)
                        photo_file.close()
                    except:
                        print('Error opening output file. ' + directory_name + '_' + band + '_' + epoch_idx)
                    
                else:
                    print("No source found in" + directory_name + '_' + band + '_' + epoch_idx)
                    
                
            except:
                print("Error making photometry " + directory_name + '_' + band + '_' + epoch_idx)
                
            
        
    # label date
    date_label = 'Date : ' + date + '  JD : ' + str(jd).strip() + '   Band : ' + band  
    fig.add_label(0.02, 0.95, date_label, relative=True, 
                  color=(0,1,1,1),
                  fontsize=9, horizontalalignment='left')


    
    outfig_name = directory_name + '_' + band + '_' + epoch_idx + '.png'
    
    fig.save(outfig_name)    
    os.system('mv ' + outfig_name + ' ' + output_path_dict[band] )
    
    # close FITS image
    hdulist.close()
    
f.close()

Will output photometry. Please define the output filename if not yet.
INFO: Auto-setting vmin to  7.805e+02 [aplpy.core]
INFO: Auto-setting vmax to  1.024e+03 [aplpy.core]
3488473_V_009
INFO: Auto-setting vmin to  1.753e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.897e+03 [aplpy.core]
3488473_R_010
INFO: Auto-setting vmin to  2.761e+01 [aplpy.core]
INFO: Auto-setting vmax to  4.249e+02 [aplpy.core]
3488473_V_066
INFO: Auto-setting vmin to  2.490e+03 [aplpy.core]
INFO: Auto-setting vmax to  3.603e+03 [aplpy.core]
3488473_I_020
INFO: Auto-setting vmin to  9.257e+02 [aplpy.core]
INFO: Auto-setting vmax to  1.764e+03 [aplpy.core]
3488473_R_040
Error making photometry 3488473_R_040


INFO: Auto-setting vmin to  1.137e+03 [aplpy.core]
INFO: Auto-setting vmax to  1.664e+03 [aplpy.core]
3488473_I_077
INFO: Auto-setting vmin to  2.597e+03 [aplpy.core]
INFO: Auto-setting vmax to  3.782e+03 [aplpy.core]
3488473_R_025
INFO: Auto-setting vmin to  1.016e+03 [aplpy.core]
INFO: Auto-setting vmax to  1.538e+03 [aplpy.core]
3488473_I_005
INFO: Auto-setting vmin to  1.360e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.142e+03 [aplpy.core]
3488473_R_031
INFO: Auto-setting vmin to  9.878e+02 [aplpy.core]
INFO: Auto-setting vmax to  1.740e+03 [aplpy.core]
3488473_R_064
INFO: Auto-setting vmin to  1.197e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.056e+03 [aplpy.core]
3488473_I_038
INFO: Auto-setting vmin to  1.144e+03 [aplpy.core]
INFO: Auto-setting vmax to  1.768e+03 [aplpy.core]
3488473_I_032
INFO: Auto-setting vmin to  9.684e+02 [aplpy.core]
INFO: Auto-setting vmax to  1.653e+03 [aplpy.core]
3488473_I_068
INFO: Auto-setting vmin to  8.979e+02 [aplpy.core]
INFO: Auto-setting vm

/home/hyliu/.conda/envs/terada2019/lib/python3.7/site-packages/aplpy/core.py:249: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  self._figure = plt.figure(**kwargs)


INFO: Auto-setting vmin to  1.023e+03 [aplpy.core]
INFO: Auto-setting vmax to  1.640e+03 [aplpy.core]
3488473_I_071


/home/hyliu/.conda/envs/terada2019/lib/python3.7/site-packages/aplpy/core.py:249: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  self._figure = plt.figure(**kwargs)


...

INFO: Auto-setting vmin to  2.030e+03 [aplpy.core]
INFO: Auto-setting vmax to  3.192e+03 [aplpy.core]
3488473_I_023


/home/hyliu/.conda/envs/terada2019/lib/python3.7/site-packages/aplpy/core.py:249: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  self._figure = plt.figure(**kwargs)


INFO: Auto-setting vmin to  1.004e+03 [aplpy.core]
INFO: Auto-setting vmax to  1.750e+03 [aplpy.core]
3488473_I_041
Error making photometry 3488473_I_041


/home/hyliu/.conda/envs/terada2019/lib/python3.7/site-packages/aplpy/core.py:249: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  self._figure = plt.figure(**kwargs)


INFO: Auto-setting vmin to  3.574e+00 [aplpy.core]
INFO: Auto-setting vmax to  1.912e+02 [aplpy.core]
3488473_V_042
